In [5]:
import pandas, numpy

## functions & options


In [22]:
input_file_directory = '/Users/kja11/OneDrive - Háskóli Íslands/PhD ATG7//0 in_silico/Python/1)data_input/'

In [20]:
output_file_directory = '/Users/kja11/OneDrive - Háskóli Íslands/PhD ATG7/0 in_silico/Python/3)output/'

# ATG7 expression, ensembl. Data preparation 

In [25]:
%%time
path = input_file_directory + "xenabrower brut data/xena_surv_ATG7.tsv"
df_ori= pandas.read_csv(path, sep = "\t")
print(df_ori.shape)
df_ori.head()

(19131, 31)
Wall time: 100 ms


,sample,samples,_sample_type,_primary_site,ENSG00000197548.12,ENST00000451513.5,ENST00000435760.5,ENST00000451830.5,ENST00000460444.5,ENST00000470474.1,...,ENST00000434066.6,ENST00000478638.5,ENST00000461278.1,ENST00000460291.1,ENST00000467121.1,ENST00000414717.5,ENST00000427759.5,ENST00000446110.1,OS,OS.time
0,TARGET-20-PARUBT-40,TARGET-20-PARUBT-40,Recurrent Blood Derived Cancer - Peripheral Blood,White blood cell,4.401,-9.9660,0.3685,2.656,-0.7108,-1.0860,...,-9.9660,-9.966,-0.6193,-5.0120,-9.966,0.8568,-9.9660,-9.9660,NaN,NaN
1,TARGET-20-PATJHJ-40,TARGET-20-PATJHJ-40,Recurrent Blood Derived Cancer - Peripheral Blood,White blood cell,2.820,-9.9660,-0.1345,-1.595,-2.9320,-9.9660,...,-9.9660,-9.966,-9.9660,-3.3080,-9.966,-2.0530,-2.7270,-0.2328,NaN,NaN
2,TARGET-10-PASLZM-40,TARGET-10-PASLZM-40,Recurrent Blood Derived Cancer - Peripheral Blood,White blood cell,1.158,-9.9660,-2.6350,-1.994,-9.9660,-9.9660,...,-9.9660,-9.966,-9.9660,-3.6260,-9.966,-1.0860,-9.9660,-9.9660,NaN,NaN
3,TARGET-21-PATAIJ-42,TARGET-21-PATAIJ-42,Post treatment Blood Cancer - Blood,White blood cell,4.332,0.2642,1.0850,-1.086,-9.9660,-9.9660,...,-9.9660,-9.966,-9.9660,-1.5950,-9.966,0.3115,-0.6643,1.7230,NaN,NaN
4,TARGET-21-PASVJS-41,TARGET-21-PASVJS-41,Post treatment Blood Cancer - Bone Marrow,White blood cell,5.156,-9.9660,1.6280,-1.732,-2.8260,-0.6643,...,-0.1187,-9.966,-1.3550,0.4125,-9.966,0.2522,-9.9660,2.4310,NaN,NaN


In [6]:
# Column selection
df = df_ori[['sample',
            '_sample_type',
            '_primary_site',
            'ENSG00000197548.12',
            'ENST00000354449.7', 
            'ENST00000354956.9',
            'ENST00000446450.6']]

# We have Adrenal Gland and Adrenal gland. I need to put "G".
df = df.assign(_primary_site=df['_primary_site'].str.title())

# Column rename
df = df.rename(columns = {'sample' : 'sample',
                    '_sample_type' : 'Sample_Type', 
                    '_primary_site' : 'Primary_Site', 
                    'ENST00000354449.7': 'ATG7_1',
                    'ENST00000354956.9': 'ATG7_2',
                    'ENST00000446450.6': 'ATG7_3',
                    'ENSG00000197548.12': 'ATG7'})

# Calcul to have value from log2;  2**(ATG7(1))-0.001
df = df.assign(ATG7_total= pow(2,df['ATG7'])-0.001,
          ATG7_1tpm= pow(2,df['ATG7_1'])-0.001,
          ATG7_2tpm= pow(2,df['ATG7_2'])-0.001,
          ATG7_3tpm= pow(2,df['ATG7_3'])-0.001)

# Calcul to have log2+1 from value; 
df = df.assign(log2_p1_ATG7_total= numpy.log2(df['ATG7_total']+1),
               log2_p1_ATG7_1= numpy.log2(df['ATG7_1tpm']+1),
               log2_p1_ATG7_2= numpy.log2(df['ATG7_2tpm']+1))

# Calcul to see percentage of expression of the 3 isoforms of ATG7 
df = df.assign(ATG7_prot_tot= df['ATG7_1tpm'] + df['ATG7_2tpm'] + df['ATG7_3tpm'])

df = df.assign(ATG7_1perc= (df['ATG7_1tpm'] *100)/ df['ATG7_prot_tot'],
              ATG7_2perc= (df['ATG7_1tpm'] *100)/ df['ATG7_prot_tot'],
              ATG7_3perc= (df['ATG7_1tpm'] *100)/ df['ATG7_prot_tot'])                              

#Add columns, normalized on total
df = df.assign(ATG7_1norm= df['ATG7_1tpm']/df["ATG7_total"],
              ATG7_2norm= df['ATG7_1tpm']/df["ATG7_total"])                                   

df.head()

,sample,Sample_Type,Primary_Site,ATG7,ATG7_1,ATG7_2,ATG7_3,ATG7_total,ATG7_1tpm,ATG7_2tpm,ATG7_3tpm,ATG7_prot_tot,ATG7_1perc,ATG7_2perc,ATG7_3perc,ATG7_1norm,ATG7_2norm
0,TARGET-20-PARUBT-40,Recurrent Blood Derived Cancer - Peripheral Blood,White Blood Cell,4.401,1.3900,1.522,-1.7320,4.467655,1.855905,1.952665,3.000343e-01,4.108604,45.171175,45.171175,45.171175,0.415409,0.415409
1,TARGET-20-PATJHJ-40,Recurrent Blood Derived Cancer - Peripheral Blood,White Blood Cell,2.820,1.1180,-1.086,-9.9660,3.010892,1.664236,0.555881,-1.495113e-07,2.220117,74.961660,74.961660,74.961660,0.552739,0.552739
2,TARGET-10-PASLZM-40,Recurrent Blood Derived Cancer - Peripheral Blood,White Blood Cell,1.158,0.0158,-3.171,-3.8160,1.691748,1.007204,0.150597,7.000183e-02,1.227802,82.033074,82.033074,82.033074,0.595363,0.595363
3,TARGET-21-PATAIJ-42,Post treatment Blood Cancer - Blood,White Blood Cell,4.332,1.3450,1.975,-9.9660,4.401843,1.823465,2.301670,-1.495113e-07,4.125135,44.203762,44.203762,44.203762,0.414250,0.414250
4,TARGET-21-PASVJS-41,Post treatment Blood Cancer - Bone Marrow,White Blood Cell,5.156,1.8040,3.769,0.5069,5.195867,2.166995,3.871026,1.419994e+00,7.458015,29.055924,29.055924,29.055924,0.417061,0.417061


In [7]:
#what kind of samples are the data ? 
df['Sample_Type'].value_counts()

Primary Tumor                                        9185
Normal Tissue                                        7429
Solid Tissue Normal                                   738
Cell Line                                             433
Metastatic                                            393
Primary Solid Tumor                                   286
Primary Blood Derived Cancer - Peripheral Blood       239
Primary Blood Derived Cancer - Bone Marrow            237
Recurrent Blood Derived Cancer - Bone Marrow          104
Recurrent Tumor                                        45
Recurrent Solid Tumor                                  13
Post treatment Blood Cancer - Bone Marrow              12
Additional - New Primary                               11
Recurrent Blood Derived Cancer - Peripheral Blood       3
Post treatment Blood Cancer - Blood                     1
Additional Metastatic                                   1
Control Analyte                                         1
Name: Sample_T

In [8]:
#How many each organs are the data ? 
df['Primary_Site'].value_counts()

Brain                         1846
Lung                          1410
Breast                        1391
Skin                          1282
Kidney                        1193
Esophagus                      848
Prostate                       648
Colon                          639
Stomach                        624
Blood Vessel                   606
White Blood Cell               595
Thyroid Gland                  571
Head And Neck Region           564
Liver                          531
Adipose Tissue                 515
Ovary                          515
Blood                          444
Bladder                        435
Muscle                         396
Heart                          377
Pancreas                       350
Testis                         319
Cervix                         309
Thyroid                        279
Nerve                          278
Soft Tissue,Bone               264
Adrenal Gland                  205
Endometrium                    204
Paraganglia         

In [9]:
#Creation of two dataframe, Normal tissue and Primary tumor.
df_ATG7_Normal = df[df['Sample_Type'].str.match ('Normal Tissue')]
df_ATG7_Primary = df[df['Sample_Type'].str.match ('Primary Tumor')]

print('normal_tissue' , len(df_ATG7_Normal))
print('primary_tumors' , len(df_ATG7_Primary))

normal_tissue 7429
primary_tumors 9185


In [10]:
# create df with only Normal and Primary 
df_ATG7_NormPrim = pandas.concat([df_ATG7_Normal, df_ATG7_Primary], axis=0)
df_ATG7_NormPrim.reset_index(drop=True, inplace=True)

In [ ]:
#save to csv
path = 'ATG7/dataframes_for_input/'
df_ATG7_NormPrim.to_csv(output_file_directory+path+'df_ATG7_NormPrim.csv',sep = "\t", index=False)

## High/Low expression per Median for Isoform expression

In [18]:
# What is the median for iso 1 and iso 2? If I want to create two groups
#High >50% and low <50%

Data = [[df_ATG7_Normal, 'Normal'], [df_ATG7_Primary, 'Primary']]
isoforms = [["ATG7_1", "iso1"], ["ATG7_2", "iso2"]]

df_LoHi = [] 

for df in Data:
  for isoform in isoforms:
    xMed = df[0][isoform[0]].median()

    low = df[0].loc[df[0][isoform[0]] < xMed].copy()
    low['L/H'] = "Low_"+isoform[1]

    high = df[0].loc[df[0][isoform[0]] > xMed].copy()
    high['L/H'] = "High_"+isoform[1]

  #merge low and high
  concat = pandas.concat([low, high], axis=0)
  col = concat.pop('L/H')
  concat.insert(3, 'L/H', col)
  df_LoHi.append(concat)

normal_LoHi = df_LoHi[0]
primary_LoHi = df_LoHi[1]

In [17]:
# create df with only Normal and Primary 
df_ATG7_NormPrim_LoHi = pandas.concat([normal_LoHi, primary_LoHi], axis=0)
df_ATG7_NormPrim_LoHi.reset_index(drop=True, inplace=True)
df_ATG7_NormPrim_LoHi

,sample,Sample_Type,Primary_Site,L/H,ATG7,ATG7_1,ATG7_2,ATG7_3,ATG7_total,ATG7_1tpm,ATG7_2tpm,ATG7_3tpm,ATG7_prot_tot,ATG7_1perc,ATG7_2perc,ATG7_3perc,ATG7_1norm,ATG7_2norm
0,GTEX-13IVO-0626-SM-5LZYJ,Normal Tissue,Muscle,Low_iso2,3.530,2.202,-2.1140,-9.9660,3.649665,2.485470,2.986649e-01,-1.495113e-07,2.784135,89.272618,89.272618,89.272618,0.681013,0.681013
1,GTEX-POYW-0526-SM-2XCEY,Normal Tissue,Muscle,Low_iso2,3.296,2.354,-5.5740,-0.5332,3.435747,2.611502,2.855818e-02,6.900203e-01,3.330081,78.421590,78.421590,78.421590,0.760097,0.760097
2,GTEX-QVJO-0126-SM-3GIK4,Normal Tissue,Muscle,Low_iso2,3.257,2.057,-0.7108,-0.0725,3.400392,2.367427,6.870439e-01,9.499886e-01,4.004460,59.119767,59.119767,59.119767,0.696222,0.696222
3,GTEX-WWTW-0626-SM-4MVNS,Normal Tissue,Muscle,Low_iso2,3.007,2.370,-1.5520,-9.9660,3.175989,2.624899,4.222728e-01,-1.495113e-07,3.047172,86.142144,86.142144,86.142144,0.826482,0.826482
4,GTEX-POYW-0526-SM-5KM2H,Normal Tissue,Muscle,Low_iso2,2.945,2.121,-9.9660,-2.2450,3.120975,2.419257,-2.156992e-07,2.099539e-01,2.629210,92.014571,92.014571,92.014571,0.775161,0.775161
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16569,TCGA-VD-AA8M-01,Primary Tumor,Eye,High_iso2,3.440,1.884,1.1120,0.4967,3.567038,2.229579,1.660130e+00,1.409982e+00,5.299691,42.069972,42.069972,42.069972,0.625050,0.625050
16570,TCGA-V4-A9E9-01,Primary Tumor,Eye,High_iso2,3.318,2.064,1.4060,-0.5125,3.455730,2.373075,1.867507e+00,7.000066e-01,4.940588,48.032232,48.032232,48.032232,0.686707,0.686707
16571,TCGA-V4-A9F8-01,Primary Tumor,Eye,High_iso2,3.302,1.276,1.4010,0.3346,3.441194,1.774277,1.863877e+00,1.260028e+00,4.898182,36.223178,36.223178,36.223178,0.515599,0.515599
16572,TCGA-WC-A88A-01,Primary Tumor,Eye,High_iso2,3.234,2.325,1.0290,-2.0530,3.379584,2.587283,1.603886e+00,2.399825e-01,4.431151,58.388503,58.388503,58.388503,0.765563,0.765563


In [14]:
#save to csv
path = 'ATG7/dataframes_for_input/'
df_ATG7_NormPrim_LoHi.to_csv(output_file_directory+path+'df_ATG7_NormPrim_LoHi.csv',sep = "\t", index=False)=